# 🌾 CropVision-AI Advanced Training (Colab Edition)

This notebook runs the advanced training pipeline for the Crop Disease Identification System. 
It connects to your GitHub repository, downloads the dataset, and runs the training with live visualization of Loss and Accuracy.

**Features:**
- MobileNetV3 (Student) + ResNet50 (Teacher) Distillation
- Live Training Plots (Loss & Accuracy)
- Structured Pruning & Quantization Aware Training (QAT)
- GPU Acceleration

In [ ]:
# 1. Setup Environment

# Clone the repository (if not already cloned)
import os
if not os.path.exists('Intelligent-Crop-Disease-Identification-System-based-on-Deep-Learning'):
    !git clone https://github.com/wskeei/Intelligent-Crop-Disease-Identification-System-based-on-Deep-Learning.git
    print("Repository cloned.")
else:
    print("Repository already exists.")

# Install dependencies
!pip install torch torchvision matplotlib tqdm

# Check GPU
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Device: {device}")
if device.type == 'cpu':
    print("⚠️ Warning: Running on CPU. Go to Runtime > Change runtime type > T4 GPU for faster training.")

In [ ]:
# 2. Download Dataset

# We use the setup logic from the repo, adapted for python execution
import os

DATA_DIR = "Intelligent-Crop-Disease-Identification-System-based-on-Deep-Learning/backend/training/data"
os.makedirs(DATA_DIR, exist_ok=True)

if not os.path.exists(os.path.join(DATA_DIR, 'PlantVillage')) and len(os.listdir(DATA_DIR)) < 5:
    print("Downloading PlantVillage Dataset...")
    !wget -q https://github.com/spMohanty/PlantVillage-Dataset/archive/refs/heads/master.zip -O plantvillage.zip
    !unzip -q plantvillage.zip
    
    # Move files to expected directory structure
    # The zip contains PlantVillage-Dataset-master/raw/color/[classes]
    # We want them in DATA_DIR/[classes]
    print("Organizing dataset...")
    !cp -r PlantVillage-Dataset-master/raw/color/* "{DATA_DIR}"
    
    # Cleanup
    !rm -rf PlantVillage-Dataset-master plantvillage.zip
    print("Dataset ready.")
else:
    print("Dataset appears to be present.")

In [ ]:
# 3. Import Modules

import sys
import time
import copy
import matplotlib.pyplot as plt
from IPython.display import clear_output
from tqdm.notebook import tqdm
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

# Add the training directory to sys.path so we can import local modules
TRAINING_PATH = os.path.abspath("Intelligent-Crop-Disease-Identification-System-based-on-Deep-Learning/backend/training")
if TRAINING_PATH not in sys.path:
    sys.path.append(TRAINING_PATH)

# Now we can import from the repo's code
from models import get_student_model, get_teacher_model
from distillation import KnowledgeDistillationLoss
from pruning import apply_structured_pruning, remove_pruning_reparameterization
from quantization import prepare_model_for_qat, convert_qat_model

print("Modules imported successfully.")

In [ ]:
# 4. Define Visualization Helper

class TrainingVisualizer:
    def __init__(self):
        self.train_losses = []
        self.val_accs = []
        self.epochs = []
        
    def update(self, epoch, train_loss, val_acc):
        self.epochs.append(epoch)
        self.train_losses.append(train_loss)
        self.val_accs.append(val_acc)
        
        clear_output(wait=True)
        fig, ax1 = plt.subplots(figsize=(10, 5))
        
        color = 'tab:red'
        ax1.set_xlabel('Epoch')
        ax1.set_ylabel('Loss', color=color)
        ax1.plot(self.epochs, self.train_losses, color=color, label='Train Loss', marker='o')
        ax1.tick_params(axis='y', labelcolor=color)
        ax1.grid(True, alpha=0.3)
        
        ax2 = ax1.twinx()  
        color = 'tab:blue'
        ax2.set_ylabel('Accuracy (%)', color=color)
        ax2.plot(self.epochs, self.val_accs, color=color, label='Val Acc', marker='s')
        ax2.tick_params(axis='y', labelcolor=color)
        
        plt.title('Training Progress')
        plt.tight_layout()
        plt.show()

In [ ]:
# 5. Configuration & Training Loop

# Config
BATCH_SIZE = 32
LEARNING_RATE = 0.001
EPOCHS_DISTILLATION = 15
PRUNE_AMOUNT = 0.2
DEBUG_MODE = False # Set to True for a quick 1-epoch test

if DEBUG_MODE:
    EPOCHS_DISTILLATION = 1
    
# Data Transforms
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

print("Loading Data...")
full_dataset = datasets.ImageFolder(DATA_DIR, transform=transform)
num_classes = len(full_dataset.classes)
print(f"Classes: {num_classes}")

train_size = int(0.8 * len(full_dataset))
val_size = len(full_dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(full_dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)

# Initialize Models
student = get_student_model(num_classes).to(device)
teacher = get_teacher_model(num_classes).to(device) # We use ImageNet weights for teacher here

criterion = KnowledgeDistillationLoss(alpha=0.5, temperature=3.0)
optimizer = optim.Adam(student.parameters(), lr=LEARNING_RATE)
viz = TrainingVisualizer()

def train_one_epoch(model, teacher, loader):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    
    pbar = tqdm(loader, desc="Training", leave=False)
    for images, labels in pbar:
        images, labels = images.to(device), labels.to(device)
        
        optimizer.zero_grad()
        student_logits = model(images)
        with torch.no_grad():
            teacher_logits = teacher(images)
            
        loss = criterion(student_logits, teacher_logits, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        _, predicted = student_logits.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()
        
        pbar.set_postfix({'loss': loss.item()})
        
    return running_loss / len(loader), 100. * correct / total

def validate(model, loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()
    return 100. * correct / total

# --- Phase 1: Distillation Training ---
print("Starting Distillation Training...")
best_acc = 0.0
for epoch in range(EPOCHS_DISTILLATION):
    loss, train_acc = train_one_epoch(student, teacher, train_loader)
    val_acc = validate(student, val_loader)
    
    viz.update(epoch + 1, loss, val_acc)
    
    if val_acc > best_acc:
        best_acc = val_acc
        torch.save(student.state_dict(), "student_best.pth")
        
print(f"Distillation Complete. Best Accuracy: {best_acc:.2f}%")

In [ ]:
# 6. Pruning & Finetuning

print("Applying Structured Pruning...")
student.load_state_dict(torch.load("student_best.pth"))
apply_structured_pruning(student, amount=PRUNE_AMOUNT)

# Finetune
print("Fine-tuning Pruned Model...")
optimizer = optim.Adam(student.parameters(), lr=LEARNING_RATE * 0.1)
for epoch in range(3):
    loss, _ = train_one_epoch(student, teacher, train_loader)
    val_acc = validate(student, val_loader)
    print(f"Pruning FT Epoch {epoch+1} - Val Acc: {val_acc:.2f}%")

remove_pruning_reparameterization(student)
torch.save(student.state_dict(), "student_pruned.pth")

In [ ]:
# 7. Quantization Aware Training (QAT)

print("Starting QAT...")
# QAT typically requires CPU for some backend configs in PyTorch, or careful specific CUDA setup
# We will stick to the script's strategy: prepare on CPU/appropriate device

student.to('cpu')
student.train()
prepare_model_for_qat(student)

optimizer = optim.Adam(student.parameters(), lr=LEARNING_RATE * 0.01)
criterion_ce = torch.nn.CrossEntropyLoss()

# Simple QAT loop
for epoch in range(3):
    student.train()
    running_loss = 0.0
    pbar = tqdm(train_loader, desc=f"QAT Epoch {epoch+1}", leave=True)
    for images, labels in pbar:
        optimizer.zero_grad()
        outputs = student(images)
        loss = criterion_ce(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        pbar.set_postfix({'loss': loss.item()})
        
quantized_model = convert_qat_model(student)
print("QAT Complete.")

In [ ]:
# 8. Save & Download
torch.save(quantized_model.state_dict(), "quantized_mobilenet.pth")

from google.colab import files
files.download('quantized_mobilenet.pth')